# Sanity check
## 1. Cel sprawozdania
Celem tego pliku jest dokumentacja kodu oraz przebadanie poprawności naszej symulacji - Przychodnia w systemie bez umówień - poprzez porównanie z teoretycznym, spodziewanym wynikiem. 
Na końcu znajdują się wyniki analizy statystycznej po wielokrotnym uruchomieniu symulacji dla prostego przykładu.
## 2. Dokumentacja kodu

In [391]:
import matplotlib
matplotlib.use('TkAgg')  # or 'Qt5Agg'
import numpy as np
import pandas as pd
import simpy
import seaborn
import matplotlib.pyplot as plt

Każdy obiekt w naszej symulacji jest klasą, która przechowuje informacje o sobie.

Dla pacjenta są to:
- `id`: identyfikator Pacjenta
- `arrival_time`: Godzina, o której przyszedł
- `service_start_time`: Godzina, o której wszedł do gabinetu (rozpoczęła się obsługa)
- `service_end_time`: Godzina, o której wyszedł z gabinetu (zakończenie obsługi)

Każda zmienna **`time`** jest przechowywana jako liczba minut od rozpoczęcia symulacji

In [392]:
class Pacjent:
    def __init__(self, id):
        self.day = 1
        self.id = id
        self.room = None
        self.arrival_time = None
        self.service_start_time = None
        self.service_end_time = None

Dla gabinetu są to:
- `id`: identyfikator gabinetu
- `env`: środowisko symulacji
- `patients_served`: liczba obsłużonych pacjentów

In [393]:
class Gabinet:
    def __init__(self, id):
        self.id = id
        self.patients_served = 0

## Parametry kliniki:
- **`number_of_ rooms`**: liczba gabinetów
- **`service_time`**: Jak długo obsługiwany jest pacjent [minuty]
- **`lambda_per_hour`**: Ile pacjentów przychodzi co godzinę (dzieli się przez 60 bo zamieniamy na minuty)
- **`sim_time`**: Długość symulacji

## Pomocnicze zmienne:
- **`queue_array`**: Kolejka (podgląd czekających pacjentów)
- **`seed`**: seed generatora liczb pseudolosowych
- **`list_rooms`**: lista gabinetów o długości **`number_of_rooms`**
- **`rooms`**: struktura Store z biblioteki simpy o wielkości **`number_of_rooms`**
- **`processed_patients`**: lista obsłużonych już pacjentów

In [394]:
class Clinic:
    def __init__(self, number_of_rooms, service_time, queue_array=[], lambda_per_hour=6/60, seed=None, sim_time=120):
        self.curr_patient_id = 1
        self.env = simpy.Environment()
        self.service_time = service_time
        self.lambda_per_hour = lambda_per_hour
        self.seed = seed
        self.list_rooms = [Gabinet(id=i + 1) for i in range(number_of_rooms)]
        self.rooms = simpy.Store(self.env, capacity=number_of_rooms)
        self.sim_time = sim_time
        self.queue_array = queue_array
        for room in self.list_rooms:
            self.rooms.put(room)
        self.processed_patients = []


### Metoda `czas()`:
Pomocnicza funkcja do ładnego wypisywania czasu.

Parametr `time_in_minutes`: czas w minutach

Praca zaczyna się od godziny 9:00 dlatego hours = 9+[...]

In [395]:

def czas(self, time_in_minutes):
    hours = 9+time_in_minutes//60
    minutes = time_in_minutes%60
    if len(str(int(minutes)))==1:
        return f"{int(hours)}:0{int(minutes)}"
    return f"{int(hours)}:{int(minutes)}"
Clinic.czas = czas

### Metoda `time_between_new_patients()`:
Ważna metoda zwracająca przerwę w minutach, do pojawienia się następnego pacjenta, podawana jest jako parametr metody `env.timeout()` po wygenerowaniu pacjenta.

Wykorzystuje rozkład wykładniczy o intensywności $\frac{1}{\lambda}$

In [396]:
def time_between_new_patients(self):
    if self.seed:
        np.random.seed(self.seed)
    return np.random.exponential(1/self.lambda_per_hour)
Clinic.time_between_new_patients = time_between_new_patients

### Metoda `generate_patients()`:

Metoda wygeneruje pacjenta o jakimś id i rozpocznie dla niego proces `serve_patient()`, aż do momentu {service_time} przed zamknięciem (Uznaliśmy, że pacjenci nie przychodzą np. 15 minut przed zamknięciem). Dzięki niej upłynie czas do przyjścia nastepnego pacjenta

In [397]:
def generate_patients(self):
    while True:
        if self.env.now >= self.sim_time-self.service_time: # Pacjenci nie przychodzą {service_time} przed zamknięciem
            break
        patient = Pacjent(id=self.curr_patient_id)
        patient.arrival_time = self.env.now
        print(f"Czas {self.czas(np.trunc(self.env.now))}: Pacjent {patient.id} przybył do kliniki")
        self.env.process(self.serve_patient(patient))
        self.curr_patient_id += 1
        yield self.env.timeout(self.time_between_new_patients())
Clinic.generate_patients = generate_patients

### Metoda `serve_patient(patient)`

Odpalana dla jakiegoś pacjenta:
- Sprawdza czy jakiś pokój jest wolny i dodaje go do podglądu kolejki (queue_array) (Taka prawdziwa kolejka jest wbudowana w SimPy i uruchamiana jest przez `rooms.get()`)
- Jeśli mija godzina *{service_time}* od końca pracy (sim_time) wtedy pacjenci są usuwani z kolejki.
- Obsługuje pacjentów i puszcza czas dalej.

In [398]:
def serve_patient(self, patient): 
    room = yield self.rooms.get()
    self.queue_array.append(patient.id)
    if self.env.now >= self.sim_time-self.service_time: # Nie zdążyło obsłużyć pacjenta przed zamknięciem
        print(f"Czas {self.czas(np.trunc(self.env.now))}: Pacjent {patient.id} - nie zdążył zostać obsłużony przed zamknięciem kliniki")
        yield self.rooms.put(room)
        return
    patient.service_start_time = self.env.now
    print(f"Czas {self.czas(np.trunc(self.env.now))}: Pacjent {patient.id} wchodzi do gabinetu {room.id} ")
    self.queue_array.remove(patient.id)
    yield self.env.timeout(self.service_time)

    patient.service_end_time = self.env.now
    room.patients_served += 1
    print(f"Czas {self.czas(np.trunc(self.env.now))}: Pacjent {patient.id} wychodzi z gabinetu {room.id}, czekal od {self.czas(np.trunc(patient.arrival_time))} do {self.czas(np.trunc(patient.service_start_time))}")
    patient.room = room.id
    self.processed_patients.append(patient)
    yield self.rooms.put(room)
Clinic.serve_patient = serve_patient

### Metoda `run()` 
Uruchamia symulacje i rozpoczyna proces generowania pacjentów.

In [399]:
def run(self, day=1):
    self.env.process(self.generate_patients())
    self.env.run(until=self.sim_time)
    df = pd.DataFrame([
    {
        "day": day,
        "id": p.id,
        "room": p.room,
        "arrival_time": p.arrival_time,
        "service_start_time": p.service_start_time,
        "service_end_time": p.service_end_time,
        "waiting_time": p.service_start_time - p.arrival_time
    }
    for p in self.processed_patients
    ])
    return df
Clinic.run = run

### Metoda `reset()`
Stworzona do resetowania informacji między symulacjami.

In [400]:
def reset(self):
    self.env = simpy.Environment()
    self.curr_patient_id = 1
    self.list_rooms = [Gabinet(id=i + 1) for i in range(len(self.list_rooms))]
    self.rooms = simpy.Store(self.env, capacity=len(self.list_rooms))
    for room in self.list_rooms:
        self.rooms.put(room)
    self.processed_patients = []
Clinic.reset = reset

### Metoda `run_multiple_times`

Tworzy nowe środowisko dla każdego uruchomienia i łączy wyniki do DataFrame'a `multiple_df`

In [401]:
def run_multiple_times(self, n):
    multiple_df = pd.DataFrame()
    for i in range(1, n+1):
        print(f"-----------{i}-----------")
        self.reset()
        single_df = self.run(i)
        multiple_df = pd.concat([multiple_df, single_df], ignore_index=True)
    return multiple_df
Clinic.run_multiple_times = run_multiple_times

## 3. Prosty przykład oraz analiza statystyczna (statystyki opisowe).

Opis zasad:
- Dla ułatwienia obliczeń pacjenci przychodzą w deterministycznych, równych odstępach czasu
- Przychodzi 6 pacjentów na godzinę (czyli przychodzą w odstępach 10 minut)
- czas trwania symulacji = 2 godziny
- czas jednej wizyty = 15 minut
- są dwa gabinety

| Czas | Zdarzenia|                              |start|koniec|                                       |
|:------:|:-----------:|------------------------------|:-----:|:------:|-----------------------------------------|
| 8:00 | Pacjent 1 wchodzi do Gabinetu 1|        |8:00|8:15|                                          |
| 8:10 | Pacjent 2 wchodzi do Gabinetu 2|        |8:10|8:25|                                          |
| 8:15 | Pacjent 1 `zwalnia` Gabinet 1 |               |-|-|                                            |
| 8:20 | Pacjent 3 wchodzi do Gabinetu 1 |        |8:20|8:35|                                         |
| 8:25 | Pacjent 2 `zwalnia` Gabinet 2  |              |-|-|                                            |
| 8:30 | Pacjent 4 wchodzi do Gabinetu 2 |        |8:30|8:45|                                         |
| 8:35 | Pacjent 3 `zwalnia` Gabinet 1   |             |-|-|                                            |
| 8:40 | Pacjent 5 wchodzi do Gabinetu 1 |       |8:40|8:55|                                          |
| 8:45 | Pacjent 4 `zwalnia` Gabinet 2   |             |-|-|                                            |
| 8:50 | Pacjent 6 wchodzi do Gabinetu 2 |      |8:50|9:05|                                           |
| 8:55 | Pacjent 5 `zwalnia` Gabinet 1   |             |-|-|                                            |
| 9:00 | Pacjent 7 wchodzi do Gabinetu 1 |       |9:00|9:15|                                          |
| 9:05 | Pacjent 6 `zwalnia` Gabinet 2   |             |-|-|                                            |
| 9:10 | Pacjent 8 wchodzi do Gabinetu 2 |       |9:10|9:25|                                          |
| 9:15 | Pacjent 7 `zwalnia` Gabinet 1  |              |-|-|                                            |
| 9:20 | Pacjent 9 wchodzi do Gabinetu 1  |      |9:20|9:35|                                          |
| 9:25 | Pacjent 8 `zwalnia` Gabinet 2   |             |-|-|                                            |
| 9:30 | Pacjent 10 wchodzi do Gabinetu 2 |       |9:30|9:45|                                         |
| 9:35 | Pacjent 9 `zwalnia` Gabinet 1    |            |-|-|                                            |
| 9:40 | Pacjent 11 wchodzi do Gabinetu 1  |       |9:40|9:55|                                        |
| 9:45 | Pacjent 10 `zwalnia` Gabinet 2    |           |-|-|                                            |
| 9:45 | Koniec przyjmowania pacjentów.   |          |-|-|                                            |
| 9:55 | Pacjent 11 `zwalnia` Gabinet 1   |            |-|-|                                            |
| 10:00 | Koniec symulacji          |                |-|-|                                            |

Podczas tej symulacji 

In [402]:
clinic = Clinic(number_of_rooms=2, service_time=15)

In [403]:
# single_df = clinic.run(day=1)
multiple_df = clinic.run_multiple_times(100)

-----------1-----------
Czas 9:00: Pacjent 1 przybył do kliniki
Czas 9:00: Pacjent 1 wchodzi do gabinetu 1 
Czas 9:00: Pacjent 2 przybył do kliniki
Czas 9:00: Pacjent 2 wchodzi do gabinetu 2 
Czas 9:08: Pacjent 3 przybył do kliniki
Czas 9:09: Pacjent 4 przybył do kliniki
Czas 9:09: Pacjent 5 przybył do kliniki
Czas 9:12: Pacjent 6 przybył do kliniki
Czas 9:14: Pacjent 7 przybył do kliniki
Czas 9:15: Pacjent 1 wychodzi z gabinetu 1, czekal od 9:00 do 9:00
Czas 9:15: Pacjent 3 wchodzi do gabinetu 1 
Czas 9:15: Pacjent 2 wychodzi z gabinetu 2, czekal od 9:00 do 9:00
Czas 9:15: Pacjent 4 wchodzi do gabinetu 2 
Czas 9:20: Pacjent 8 przybył do kliniki
Czas 9:30: Pacjent 3 wychodzi z gabinetu 1, czekal od 9:08 do 9:15
Czas 9:30: Pacjent 5 wchodzi do gabinetu 1 
Czas 9:30: Pacjent 4 wychodzi z gabinetu 2, czekal od 9:09 do 9:15
Czas 9:30: Pacjent 6 wchodzi do gabinetu 2 
Czas 9:35: Pacjent 9 przybył do kliniki
Czas 9:37: Pacjent 10 przybył do kliniki
Czas 9:42: Pacjent 11 przybył do kliniki
Cz

In [404]:
multiple_df.head()

,day,id,room,arrival_time,service_start_time,service_end_time,waiting_time
0,1,1,1,0.000000,0.000000,15.000000,0.000000
1,1,2,2,0.615976,0.615976,15.615976,0.000000
2,1,3,1,8.387531,15.000000,30.000000,6.612469
3,1,4,2,9.151688,15.615976,30.615976,6.464288
4,1,5,1,9.699070,30.000000,45.000000,20.300930


In [405]:
multiple_df.describe()

,day,id,room,arrival_time,service_start_time,service_end_time,waiting_time
count,1022.000000,1022.000000,1022.000000,1022.000000,1022.000000,1022.000000,1022.000000
mean,49.529354,5.907045,1.478474,44.524084,48.822380,63.822380,4.298296
std,30.016330,3.401931,0.499781,31.327371,32.418270,32.418270,7.519137
min,1.000000,1.000000,1.000000,0.000000,0.000000,15.000000,0.000000
25%,22.250000,3.000000,1.000000,16.155135,18.724511,33.724511,0.000000
50%,49.000000,6.000000,1.000000,43.704479,48.300803,63.300803,0.000000
75%,77.000000,8.000000,2.000000,71.760501,76.380959,91.380959,6.625914
max,100.000000,14.000000,2.000000,104.848422,104.970357,119.970357,54.712473
